# Combinando conjuntos de datos: concat y append

Algunos de los estudios de datos más interesantes provienen de la combinación de diferentes fuentes de datos.
Estas operaciones pueden implicar cualquier cosa, desde una concatenación muy sencilla de dos conjuntos de datos diferentes hasta uniones y fusiones más complicadas al estilo de una base de datos que manejan correctamente cualquier superposición entre los conjuntos de datos.
`Series` y ``DataFrame`` se construyen con este tipo de operación en mente, y Pandas incluye funciones y métodos que hacen que este tipo de procesamiento de datos sea rápido y sencillo.

Aquí veremos la concatenación simple de `Series` y ``DataFrame`` con la función `pd.concat`; Más adelante profundizaremos en fusiones y uniones en memoria más sofisticadas implementadas en Pandas.

Comenzamos con las importaciones estándar:

In [1]:
import pandas as pd
import numpy as np

Por conveniencia, definiremos esta función, que crea un `DataFrame` de una forma particular que será útil en los siguientes ejemplos:

In [2]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


Además, crearemos una clase rápida que nos permita mostrar múltiples ``DataFrame`` uno al lado del otro. El código hace uso del método especial `_repr_html_`, que IPython/Jupyter utiliza para implementar su visualización enriquecida de objetos:

In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

El uso de esto quedará más claro a medida que continuemos nuestra discusión en la siguiente sección.

## Recordar: Concatenación de matrices NumPy

La concatenación de objetos `Series` y `DataFrame` se comporta de manera similar a la concatenación de matrices NumPy, que se puede realizar mediante la función `np.concatenate`, como se analiza en [Los conceptos básicos de las matrices NumPy] (02.02-Los-conceptos-básicos-de- NumPy-Arrays.ipynb).
Recuerde que con él puede combinar el contenido de dos o más matrices en una sola matriz:

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

El primer argumento es una lista o tupla de matrices para concatenar.
Además, en el caso de matrices multidimensionales, se necesita una palabra clave `axis` que le permite especificar el eje a lo largo del cual se concatenará el resultado:

In [5]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## Concatenación simple con pd.concat

La función `pd.concat` proporciona una sintaxis similar a `np.concatenate` pero contiene una serie de opciones que discutiremos en un momento:

```pitón
# Firma en Pandas v1.3.5
pd.concat(objs, eje=0, unirse='exterior', ignore_index=False, claves=Ninguno,
          niveles=Ninguno, nombres=Ninguno, verificar_integridad=Falso,
          ordenar=Falso, copiar=Verdadero)
```

`pd.concat` se puede usar para una concatenación simple de objetos `Series` o `DataFrame`, del mismo modo que `np.concatenate` se puede usar para concatenaciones simples de matrices:

In [6]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

También funciona para concatenar objetos de dimensiones superiores, como ``DataFrame``s:

In [7]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Su comportamiento predeterminado es concatenar filas dentro del `DataFrame` (es decir, `axis=0`).
Al igual que `np.concatenate`, `pd.concat` permite especificar un eje a lo largo del cual se llevará a cabo la concatenación.
Considere el siguiente ejemplo:

In [8]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='columns')")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis='columns')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

Podríamos haber especificado de manera equivalente ``axis=1``; aquí hemos utilizado el método más intuitivo ``axis='columnas'``.

### Índices duplicados

Una diferencia importante entre `np.concatenate` y `pd.concat` es que la concatenación de Pandas *conserva los índices*, ¡incluso si el resultado tendrá índices duplicados!
Considere este breve ejemplo:

In [9]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make indices match
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Observe los índices repetidos en el resultado.
Si bien esto es válido dentro de los ``DataFrame``s, el resultado suele ser indeseable.
`pd.concat` nos brinda algunas formas de manejarlo.

#### Tratar los índices repetidos como un error

Si desea simplemente verificar que los índices en el resultado de `pd.concat` no se superpongan, puede incluir el indicador `verify_integrity`.
Con esto establecido en "Verdadero", la concatenación generará una excepción si hay índices duplicados.
Aquí hay un ejemplo, donde para mayor claridad capturaremos e imprimiremos el mensaje de error:

In [10]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### Ignorando el índice

A veces el índice en sí no importa y usted preferiría simplemente ignorarlo.
Esta opción se puede especificar usando el indicador `ignore_index`.
Con esto establecido en "True", la concatenación creará un nuevo índice entero para el "DataFrame" resultante:

In [11]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### Agregar claves MultiIndex

Otra opción es utilizar la opción `keys` para especificar una etiqueta para las fuentes de datos; el resultado será una serie indexada jerárquicamente que contiene los datos:

In [12]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

Podemos usar las herramientas analizadas en [Indexación jerárquica] (03.05-Hierarchical-Indexing.ipynb) para transformar este `DataFrame` indexado múltiples veces en la representación que nos interesa.

### Concatenación con uniones

En los breves ejemplos que acabamos de ver, principalmente estábamos concatenando ``DataFrame``s con nombres de columnas compartidos.
En la práctica, los datos de diferentes fuentes pueden tener diferentes conjuntos de nombres de columnas y `pd.concat` ofrece varias opciones en este caso.
Considere la concatenación de los siguientes dos ``DataFrame``s, que tienen algunas (¡pero no todas!) columnas en común:

In [13]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

El comportamiento predeterminado es completar las entradas para las que no hay datos disponibles con valores NA.
Para cambiar esto, podemos ajustar el parámetro `join` de la función `concat`.
Por defecto, la unión es una unión de las columnas de entrada (`join='outer'`), pero podemos cambiar esto a una intersección de las columnas usando `join='inner'`:

In [14]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

Otro patrón útil es utilizar el método `reindex` antes de la concatenación para tener un control más preciso sobre qué columnas se eliminan:

In [15]:
pd.concat([df5, df6.reindex(df5.columns, axis=1)])

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


### El método de agregar

Debido a que la concatenación directa de matrices es tan común, los objetos `Series` y `DataFrame` tienen un método `append` que puede lograr lo mismo con menos pulsaciones de teclas.
Por ejemplo, en lugar de `pd.concat([df1, df2])`, puede usar `df1.append(df2)`:

In [16]:
display('df1', 'df2', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Tenga en cuenta que, a diferencia de los métodos `append` y `extend` de las listas de Python, el método `append` en Pandas no modifica el objeto original; en su lugar, crea un nuevo objeto con los datos combinados.
Tampoco es un método muy eficiente, porque implica la creación de un nuevo índice *y* búfer de datos.
Por lo tanto, si planea realizar múltiples operaciones de "agregar", generalmente es mejor crear una lista de objetos "DataFrame" y pasarlos todos a la vez a la función "concat".

En el próximo capítulo, veremos un enfoque más poderoso para combinar datos de múltiples fuentes: las fusiones/uniones estilo base de datos implementadas en `pd.merge`.
Para obtener más información sobre `concat`, `append` y funciones relacionadas, consulte la sección ["Fusionar, unir, concatenar y comparar"](http://pandas.pydata.org/pandas-docs/stable/merging.html ) de la documentación de Pandas.